<a href="https://colab.research.google.com/github/aaron020/CNN_Project/blob/main/resNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Based on tutorial from : https://www.youtube.com/watch?v=glmowUlqoYw

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os 

In [2]:
#pull in a dataset from your computer
pathName = "Shoe vs Sandal vs Boot Dataset"


footwear_types = os.listdir(pathName)

print("The types of footwear are = " , footwear_types)

The types of footwear are =  ['Shoe', 'Sandal', 'Boot']


In [3]:
#Tack all the data from the datset 
footwear = []

for item in footwear_types:
  # get all the files from each folder 
  all_footwear = os.listdir(pathName + '/' + item)

  for fw in all_footwear:
    footwear.append((item, str(pathName + '/' + item) + '/' + fw))

print(footwear[0])

('Shoe', 'Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (4002).jpg')


In [4]:
#Store the folder name in a seperate column - the folder name is what the image is

#Build a dataframe using pandas

footwear_df = pd.DataFrame(data=footwear, columns=['footwear type', 'image'])

print(footwear_df.head())

  footwear type                                              image
0          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (4002...
1          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (388)...
2          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (4868...
3          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (3740...
4          Shoe  Shoe vs Sandal vs Boot Dataset/Shoe/Shoe (1919...


In [5]:
#Print info about the dataset

print('Total number of rooms in the dataset: ', len(footwear))

footwear_count = footwear_df['footwear type'].value_counts()

print('Rooms in each category')
print(footwear_count)

Total number of rooms in the dataset:  15000
Rooms in each category
Shoe      5000
Sandal    5000
Boot      5000
Name: footwear type, dtype: int64


In [6]:
import cv2
path = 'Shoe vs Sandal vs Boot Dataset/'

#Original image size = 136x102, now we resize to 244x244

img_size = 244


images = []
labels = []

for i in footwear_types:
  data_path = path + str(i) #enter into first folder and second and then third 
  filenames = [i for i in os.listdir(data_path)] # name of each image 

  for f in filenames:
    img = cv2.imread(data_path + '/' + f)
    img = cv2.resize(img, (img_size, img_size))
    images.append(img)
    labels.append(i)

In [7]:
#Transform the image array to a numpy type 
images = np.array(images)
images.shape 
#Shape ->  total No. of images x size x dimensions (RGB)

(15000, 244, 244, 3)

In [8]:
images = images.astype(np.float16) / 255.0
#np.float16 or it will run out of ram
#ie divide all the pixel values by 255 just to make the pixel value smaller 
#255 = max pixel value everythng else is gonna be like 0.13 0.43

In [9]:
#The labels are in the form of numbers, the computer does not understand that so we convert 
#them to numbers

from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
y = footwear_df['footwear type'].values
y = y.reshape(-1, 1)

onehotencoder = OneHotEncoder()
Y = onehotencoder.fit_transform(y)
print(Y.shape)
print(Y.toarray())



(15000, 3)
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [10]:
#Divide datset into train and test

from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.10, random_state=415)
#Very small test set as the dataset is very small

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)



(13500, 244, 244, 3)
(13500, 3)
(1500, 244, 244, 3)
(1500, 3)


In [12]:
##Start of building RESNET functions


from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image 
from keras.utils import layer_utils 
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform 
import scipy.misc
from matplotlib.pyplot import imshow


In [13]:
def identity_block (X, f, filters):
  F1, F2, F3 = filters

  #Input value is saved so that it can be applied after conv layers
  X_shortcut = X


  #First layer                                                                  X is the input 
  X = Conv2D(filters = F1, kernal_size= (1,1), strides= (1,1), padding = 'same')(X)

  #X is the input to batch normalization layer 
  X = BatchNormalization(axis = 3)(X)

  #Relu is the activation function used 
  X = Activation('relu')(X)



  #Secound layer 
  X = Conv2D(filters = F2, kernal_size= (f,f), strides= (1,1), padding = 'same')(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)

  #Third layer 
  X = Conv2D(filters = F3, kernal_size= (1,1), strides= (1,1), padding = 'valid')(X)
  X = BatchNormalization(axis = 3)(X)
  #No relu here instead we push it to be added to the first input

  #Add the shortcut value to F(x)
  X = Add()([X, X_shortcut])
  X = Activation('relu')(X) #Relu is ued because it will transform all negative pixel vals to 0

  return X


In [15]:
def convolutional_block(X,f,filters, s=2):
  F1, F2, F3 = filters

  X_shortcut = X

  #First layer - stride causes a reduction in the size of the image
  X = Conv2D(filters = F1, kernal_size = (1,1), strides = (s,s)) (X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)

  #Second layer
  X = Conv2D(filters = F2, kernal_size= (1,1), strides = (1,1), padding = 'same') (X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)

  
  #Third layer
  X = Conv2D(filters = F3, kernal_size= (1,1), strides = (1,1), padding = 'valid')(X)
  X = BatchNormalization(axis = 3)(X)

  #The output now has a different size to the input 
  #Thus we must make a change to the shoutcut
  #Notice this conv layer is almost the same as the first layer (the one that reduced the size)
  X_shortcut = Conv2D(filters = F3, kernal_size= (1,1), strides = (s,s), padding = 'valid')(X_shortcut)
  X_shortcut = BatchNormalization(axis = 3)(X_shortcut)

  X = add()([X, X_shortcut])
  X = Activation('relu')(X)

  return x


In [23]:
#Resnet algorithm  - all 50 layers


def Resnet50(input_shape=(224,224,3), classes=3):

  #Define the input with shape input_shape 
  X_input = Input(input_shape)

  #zero padding 
  X = ZeroPadding2D((3,3))(X_input) #3x3 padding, thus increasing the size 
  #becaue the size of the image can become too small without the padding 
  #Allows use to preserve the information - optional 

  #Stage 1
  X = Conv2D(64, (7,7), strides=(2,2))(X)
  X = BatchNormalization(axis = 3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3,3), strides=(2,2))(X)


  #Stage 2
  X = convolutional_block(X, f=3, filters=[64,64,256], s=1)

  X = identity_block(X, 3, [64,64,256])

  X = identity_block(X, 3, [64,64,256])


  #Stage 3
  X = convolutional_block(X, f=3, filters=[128,128,512], s=2)

  X = identity_block(X, 3, [128,128,512])  

  X = identity_block(X, 3, [128,128,512])  

  X = identity_block(X, 3, [128,128,512])  

  #Stage 4
  X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)

  X = identity_block(X, 3, [256,256,1024])

  X = identity_block(X, 3, [256,256,1024])

  X = identity_block(X, 3, [256,256,1024])

  X = identity_block(X, 3, [256,256,1024])

  X = identity_block(X, 3, [256,256,1024])

  #Stage 5
  X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)

  X = identity_block(X, 3, [512,512,2048])

  X = identity_block(X, 3, [512,512,2048])


  #Average pooling layer 
  X = AveragePooling2D((2,2),name="avg_pool")(X)


  #Output layer 
  #Convert image to a 1 dimensional vector 
  X = Flatten()(X)
  #name = 'fc' means nothing 
  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernal_initializer = glorot_uniform(seed=0))(X)

  #Create the model 
  model = Model(inputs = X_input, outputs = X, name = 'Resnet50')

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  model.summary()
  return model



In [22]:
model = Resnet50()

TypeError: ignored

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_x, train_y, epochs = 3, batch_size = 32)

In [ ]:
model.evaluate(test_x, test_y)